# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [13]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( 'mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

### 10 countries with the lowest infant mortality rates

In [6]:
document = ET.parse( 'mondial_database.xml' )
root = document.getroot()
tcountry = []
tinfant_mortality = []
for country in root.iter('country'):
    if country.find('name') != None:
        if country.find('infant_mortality') != None:
            tcountry.append(country.find('name').text)
            tinfant_mortality.append(float(country.find('infant_mortality').text))
        else:
            continue
    else:
        continue

datalist = {'country':tcountry, 'infant mortality': tinfant_mortality}
dframe = pd.DataFrame(datalist).sort_values(by='infant mortality').head(10)
dframe

,country,infant mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### 10 cities with the largest population

In [7]:
cityname = ""
citypopulation = ""
document = ET.parse( 'mondial_database.xml' )
df = pd.DataFrame(columns=['CityName','Population']) #create data frame to hold country name and its popuplation
df['Population'] = df['Population'].astype(float)

#loop through country element to find city name and its population
for country in document.iterfind( 'country' ):
    for city in country.iter('city'): #find all cities within each country element
        cityname = city.find('name').text
        year = int(0)
        for node in city.iterfind('population'): #find all population elements with each city
            year = node.attrib['year'] #there are multiple population elements with different 'year' attribute
            if node.attrib['year'] >= year: #store the population number of the latest year
                citypopulation = int(node.text)
        df.loc[len(df)] = [cityname,citypopulation] #add city name and its population to data frame
        cityname = ""
        

#sort data frame to find 10 cities with largest population
df.sort_values(by = 'Population', ascending=False).head(10)

,CityName,Population
1341,Shanghai,22315474.0
771,Istanbul,13710512.0
1527,Mumbai,12442373.0
479,Moskva,11979529.0
1340,Beijing,11716620.0
2810,São Paulo,11152344.0
1342,Tianjin,11090314.0
1064,Guangzhou,11071424.0
1582,Delhi,11034555.0
1067,Shenzhen,10358381.0


### 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [11]:
ethnic_group = []
total_population = []
ethnic_numbers = []


for country in root.findall('country'):
    for population in reversed(country.findall('population')):
        total_population.append(int(population.text))
        for ethnicity in country.findall('ethnicgroup'):
            ethnic_numbers.append((int(population.text), float(ethnicity.attrib['percentage']), ethnicity.text))
        break

d= pd.DataFrame(ethnic_numbers, columns=['population', 'percentage', 'ethnicity'])
d['Ethnic Population'] = (d.population * d.percentage)/100

d.groupby('ethnicity').sum().sort_values(by='Ethnic Population', ascending=False).head(10)

,population,percentage,Ethnic Population
ethnicity,,,
Han Chinese,1360720000,91.50,1.245059e+09
Indo-Aryan,1210854977,72.00,8.718156e+08
European,1157295639,970.82,4.948722e+08
African,975352746,1868.55,3.183251e+08
Dravidian,1210854977,25.00,3.027137e+08
Mestizo,279743964,870.70,1.577344e+08
Bengali,149772364,98.00,1.467769e+08
Russian,322438406,224.10,1.318570e+08
Japanese,127298000,99.40,1.265342e+08


### Longest River

In [14]:
name = []
length = []
source = []

document = ET.parse( 'mondial_database.xml' )


for element in document.getiterator('river'):
    if element.find('length') !=None:
        name.append(element.find('name').text)
        length.append(np.float(element.find('length').text))
        
riverList ={'name':name,
        'length':length} 
river=pd.DataFrame(riverList,columns=['name','length']).sort_values(by='length',ascending=False).head(1)
river

,name,length
174,Amazonas,6448.0


### Largest Lake

In [15]:
lakeList=[]
for element in document.getiterator('lake'):
    for subelement in list(element):
        if subelement.tag == 'name':
            Lname = subelement.text
        if subelement.tag == 'area':
            Larea = float(subelement.text)
        if subelement.tag == 'located':
            Llocation = subelement.attrib['country']
    lakeList.append((Lname, Larea, Llocation))
lake = pd.DataFrame(lakeList, columns=['Name', 'Area', 'C_Code']).sort_values(by='Area', ascending=False).head(1)
lake

,Name,Area,C_Code
54,Caspian Sea,386400.0,TM


### Airport at highest elevation

In [16]:

document = ET.parse( 'mondial_database.xml' )
doc = document.getroot()
airportList = []
for element in doc.findall('airport'):
    if (element.find('elevation').text != None and element.find('name') != None):
        airportList.append((float(element.find('elevation').text), element.find('name').text, element.get('country')))
aList = pd.DataFrame(airportList, columns=['Elevation', 'Name', 'Code']).sort_values(by='Elevation', ascending=False)
aList.head(1)

,Elevation,Name,Code
80,4063.0,El Alto Intl,BOL
